In [2]:
import numpy as np
import random
import copy

# ↓問題生成

In [48]:
def puzzle_generation(puzzle_board, issue):
    #ファイル読み込み
    f = open(issue, 'r', encoding='UTF-8')
    data = f.readlines()
    f.close()

    s0 = data[0].rstrip('\n').rstrip(',')
    l0 = [int(x.strip()) for x in s0.split(',')]

    colmun = l0[0]
    row = l0[1]
    selection_cost = l0[2]

    for i in range(1, row+1):
        row_data = data[i].rstrip('\n').rstrip(',')
        line_data = [int(x.strip()) for x in row_data.split(',')]
        row_array = np.array(line_data)
        if i == 1:
            puzzle_board = row_array
        else:
            puzzle_board = np.vstack((puzzle_board, row_array))

    #ランダム生成
    # random_num = random.sample(range(1, colmun*row+1), colmun*row)
    # random_list = np.array(random_num)
    # puzzle_board = random_list.reshape(colmun, row)

    return puzzle_board, colmun, row, selection_cost

# ↓回答書き込み

In [ ]:
def answer_writing():

# ↓移動関数

In [ ]:
#選択画像を上に移動
def up():

In [ ]:
#選択画像を下に移動
def down():

In [ ]:
#選択画像を左に移動
def left():

In [ ]:
#選択画像を右に移動
def right():

# ↓それぞれの処理

In [ ]:
# 2行を除いた盤面の並び替え(たくと)
def puzzle_board_top(puzzle_board_top):

In [ ]:
# 下2行の右4マスを除いた盤面の並び替え(たいち)
def puzzle_board_bottom_left(puzzle_board_bottom_left):

In [ ]:
# 右下4ますの並び替え(しのべー)
def puzzle_board_bottom_right(puzzle_board_bottom_right):

# メイン関数

In [49]:
puzzle_board =  np.array([])
puzzle_board, colmun, row, selection_cost = puzzle_generation(puzzle_board, 'issue/4x4.txt')

print('colmun(列→)：' + str(colmun))
print('row(行↓)：' + str(row))
print('selection_cost(選択コスト)：' + str(selection_cost) + '\n')
print(puzzle_board)

colmun(列→)：4
row(行↓)：4
selection_cost(選択コスト)：2

[[11  9  5  8]
 [12  4  7 10]
 [15  3  1 13]
 [14  2 16  6]]
